### Environment variables

In [21]:
HOME = "/home/chiajui.chou"
DEEPCLEAN_CONTAINER_ROOT = f"{HOME}/images/deepclean"
DATA_DIR = f"{HOME}/deepclean/data/CDC_test-180Hz"
RESULTS_DIR = f"{HOME}/deepclean/results"

DEEPCLEAN_IFO = "H1"
DEEPCLEAN_PROBLEM = "180Hz"
GPU_INDEX = 0

### Input to deepclean.tasks.Train

In [32]:
# luigi.cfg
cfg = dict.fromkeys(['deepclean', 'core'])
for key in cfg:
    cfg[key] = dict()

cfg['deepclean']['ifo'] = DEEPCLEAN_IFO
cfg['deepclean']['problem'] = DEEPCLEAN_PROBLEM
cfg['deepclean']['strain_channel'] = "GDS-CALIB_STRAIN"
cfg['core']['local_scheduler'] = True
cfg['core']['module'] = "deepclean"

# deepclean.config.deepclean
ifo = cfg['deepclean']['ifo']
problem = [cfg['deepclean']['problem']]
strain_channel = cfg['deepclean']['strain_channel']
container_root = DEEPCLEAN_CONTAINER_ROOT
# subtraction problems
from deepclean.couplings import subtraction_problems
couplings = [subtraction_problems[i][ifo] for i in problem]
witnesses = [j for i in couplings for j in i.channels]
freq_low = [i.freq_low for i in couplings]
freq_high = [i.freq_high for i in couplings]

# train.sh
image = "train.sif"
gpus = GPU_INDEX
data_fname = f"{DATA_DIR}/deepclean-1378402219-3072.hdf5"
train_config = f"{HOME}/deepcleanv2/projects/train/config.yaml"
output_dir = f"{RESULTS_DIR}/O4-CDC_180Hz"

# Train task
config = train_config
fname = data_fname
channels = [strain_channel] + witnesses
freq_low = freq_low
freq_high = freq_high

In [31]:
print(config)
print(fname)
print(channels)
print(freq_low)
print(freq_high)

/home/chiajui.chou/deepcleanv2/projects/train/config.yaml
/home/chiajui.chou/deepclean/data/CDC_test-180Hz/deepclean-1378402219-3072.hdf5
['GDS-CALIB_STRAIN', 'H1:PEM-CS_MAG_LVEA_OUTPUTOPTICS_Z_DQ', 'H1:LSC-REFL_A_LF_OUT_DQ', 'H1:IMC-F_OUT_DQ', 'H1:IMC-WFS_B_Q_YAW_OUT_DQ', 'H1:IMC-WFS_A_Q_YAW_OUT_DQ', 'H1:PEM-EX_MAG_VEA_FLOOR_X_DQ', 'H1:PEM-CS_ACC_PSL_PERISCOPE_Y_DQ', 'H1:ISI-ITMY_ST2_BLND_RZ_GS13_CUR_IN1_DQ', 'H1:IMC-WFS_A_DC_YAW_OUT_DQ', 'H1:IMC-WFS_B_DC_YAW_OUT_DQ', 'H1:ISI-HAM6_BLND_GS13Z_IN1_DQ', 'H1:PEM-EY_VMON_ETMY_ESDPOWERMINUS18_DQ', 'H1:ISI-HAM2_BLND_GS13RZ_IN1_DQ', 'H1:IMC-L_OUT_DQ', 'H1:PEM-CS_ACC_PSL_PERISCOPE_X_DQ', 'H1:PEM-CS_MAG_EBAY_LSCRACK_Z_DQ', 'H1:IMC-DOF_4_Y_IN1_DQ', 'H1:IMC-WFS_B_I_YAW_OUT_DQ']
[176]
[184]


In [33]:
# Check train_config
import yaml
with open(config, 'r') as file:
    config = yaml.safe_load(file)

print(config['data']['train_duration'])
print(config['data']['train_stride'])
print(config['data']['valid_frac'])
print(config['data']['inference_sampling_rate'])
print(config['data']['clean_kernel_length'])
print(config['data']['clean_stride'])

1024
0.0625
0.33
2
1
1


### projects/train/cli.py

In [36]:
import os
import torch
from lightning.pytorch.cli import LightningCLI

from train.data import DeepCleanDataset
from train.model import DeepClean
from utils.logging import configure_logging

### Data Setup

In [24]:
from train.data import DeepCleanDataset
dc_dataset = DeepCleanDataset(
    fanme=fname,
    channels=channels,
    kernel_length=kernel_length,
    freq_low=freq_low,
    freq_high=freq_high,
    batch_size=batch_size,
    train_duration=train_duration,
    test_duration=test_duration,
    valid_frac=valid_frac,
    inference_sampling_rate=inference_sampling_rate,
    clean_kernel_length=clean_kernel_length,
    clean_stride=clean_stride,
    start_offset=start_offset,
    filt_order=filt_order,
)

NameError: name 'kernel_length' is not defined